In [1]:
import logging
import os

# Import our modules
from calibration import calibrate_vix_model
from pricing import (
    GenericMCPricer,
    VIXDedicatedMCPricer,
    load_vix_data, 
    infer_current_regime, 
    create_regime_process
)

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

data_path = os.path.join('..', 'data', 'vix_data.csv')
vix_df = load_vix_data(data_path)
vix_values = vix_df['VIX'].values

logger.info("Calibrating VIX model...")
calibrated_params, log_likelihoods = calibrate_vix_model(vix_values)
# calibrated_params = initialize_vix_params(vix_values, num_regimes=2)

logger.info("Inferring current regime...")
current_regime, regime_probs = infer_current_regime(vix_values, calibrated_params)
logger.info(f"Most likely current regime: {current_regime}")
logger.info(f"Regime probabilities: {regime_probs}")

logger.info("Creating regime process...")
regime_process = create_regime_process(calibrated_params)

current_vix = vix_values[-1]
strike_price = 13.0  # ATM option
expiry = 30  # 30 days to expiration

logger.info("Done")

INFO:__main__:Calibrating VIX model...
INFO:calibration:Calibrating 2-regime VIX model
INFO:calibration:Data: 439 observations
INFO:calibration:VIX range: [11.86, 52.33]
INFO:particle_filter:EM Iteration 1/30


INFO:particle_filter:  Log-likelihood: -6169.8741
INFO:calibration:  Regime 0: kappa=20.0000, theta=0.1620, sigma=0.2674
INFO:calibration:  Regime 1: kappa=0.1000, theta=0.1654, sigma=0.1783
INFO:particle_filter:EM Iteration 2/30
INFO:particle_filter:  Log-likelihood: -1144.7926
INFO:particle_filter:  Improvement: 5025.081479
INFO:calibration:  Regime 0: kappa=20.0000, theta=0.1654, sigma=0.9974
INFO:calibration:  Regime 1: kappa=0.1000, theta=0.1721, sigma=0.4279
INFO:particle_filter:EM Iteration 3/30
INFO:particle_filter:  Log-likelihood: -1113.4140
INFO:particle_filter:  Improvement: 31.378597
INFO:calibration:  Regime 0: kappa=20.0000, theta=0.1800, sigma=1.2743
INFO:calibration:  Regime 1: kappa=0.1000, theta=0.1717, sigma=0.5370
INFO:particle_filter:EM Iteration 4/30
INFO:particle_filter:  Log-likelihood: -1118.0108
INFO:particle_filter:  Improvement: -4.596815
INFO:calibration:  Regime 0: kappa=0.1000, theta=0.1931, sigma=1.6018
INFO:calibration:  Regime 1: kappa=20.0000, theta=

In [3]:
logger.info("Creating VIX generic MC pricer...")
pricer = GenericMCPricer(regime_process)
maturity = expiry / 252.0

logger.info("Pricing VIX options...")
for c in range(9, 10):
    strike = current_vix * (c / 10)
    call_price = pricer.price_european(
        initial_val=current_vix / 100.0,
        initial_regime=current_regime,
        strike=strike,
        maturity=maturity,
        option_type='call'
    )
    put_price = pricer.price_european(
        initial_val=current_vix / 100.0,
        initial_regime=current_regime,
        strike=strike,
        maturity=maturity,
        option_type='put'
    )
    
    logger.info(f" Strike {strike:.1f}, Expiry {expiry} days:")
    logger.info(f"  Call: {call_price:.2f}")
    logger.info(f"  Put:  {put_price:.2f}")


INFO:__main__:Creating VIX generic MC pricer...
INFO:__main__:Pricing VIX options...
INFO:__main__: Strike 14.7, Expiry 30 days:
INFO:__main__:  Call: 2.21
INFO:__main__:  Put:  0.00


In [4]:
logger.info("Creating VIX dedicated MC pricer...")
pricer = VIXDedicatedMCPricer(regime_process)

logger.info("Pricing VIX options...")
for c in range(9, 12):
    strike = current_vix * (c / 10)
    call_price = pricer.price_vix_option(
        initial_vix=current_vix,
        initial_regime=current_regime,
        strike=strike,
        expiry_days=expiry,
        option_type='call'
    )
    put_price = pricer.price_vix_option(
        initial_vix=current_vix,
        initial_regime=current_regime,
        strike=strike,
        expiry_days=expiry,
        option_type='put'
    )
    
    logger.info(f"Strike {strike:.1f}, Expiry {expiry} days:")
    logger.info(f"  Call: {call_price:.2f}")
    logger.info(f"  Put:  {put_price:.2f}")

INFO:__main__:Creating VIX dedicated MC pricer...
INFO:__main__:Pricing VIX options...
INFO:__main__:Strike 14.7, Expiry 30 days:
INFO:__main__:  Call: 2.05
INFO:__main__:  Put:  0.00
INFO:__main__:Strike 16.3, Expiry 30 days:
INFO:__main__:  Call: 0.52
INFO:__main__:  Put:  0.07
INFO:__main__:Strike 17.9, Expiry 30 days:
INFO:__main__:  Call: 0.16
INFO:__main__:  Put:  1.34
